In [1]:
import json
import json
import re
import os
import dotenv
from pymongo import MongoClient

dotenv.load_dotenv("../../.env")

client = MongoClient(os.getenv('MONGO_URI'))
db = client['examined_life']
collection = db['days']



In [2]:
cloned_records = list(collection.find({}))

In [3]:
import sys

def get_size(obj, seen=None):
    """Recursively find the size of an object in bytes."""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

# Example usage
record_size = get_size(cloned_records[0])
print(f"Size of the record in bytes: {record_size}")

Size of the record in bytes: 36217


In [4]:
cloned_records[-2]

{'_id': ObjectId('679f4f737e937b273d9826b9'),
 'dayAndNightOf': '2025-01-31',
 'docRef': None,
 'timestampWritten': datetime.datetime(2025, 2, 2, 8, 24, 15, 426000),
 'ml': {'night:yasaExtended:N1:alphaabsab_s:mean': 0.6958727798666666,
  'drugsAndSupplements:GabapentinGabasignCount': 0,
  'night:yasaExtended:Statistics:SME': 78.06291390728477,
  'exercise:byExercise:Run:averageHeartRate': 0,
  'exercise:fitbit:heartRateZones:FatBurn:min': 108,
  'night:yasaExtended:NonDeep:alphaabsaa_s:mean': 0.12911766207361464,
  'night:yasaExtended:R:sigmaab_s:mean': 0.5546093605185188,
  'night:yasaExtended:R:alphaabsaa:mean': 0.00878497445148148,
  'exercise:fitbit:heartRateZones:OutofRange:minutes': 508,
  'night:yasaExtended:NonDeep:at_s:mean': 0.5911525068776364,
  'night:yasaExtended:All:sdelta_s:mean': 0.5920637788247978,
  'hrv:GotIntoBedToReadyToSleep:mean': 33.1948051948052,
  'phoneUsage:allDay:categories:NonCalming:totalDurationSecs': 12850,
  'night:yasaExtended:R:fdeltaabsaa_s:mean': 

In [23]:
!pip install pympler

In [27]:
from collections import defaultdict
from pympler import asizeof

def get_size(obj, seen=None):
    """Recursively find the size of an object in bytes."""
    size = 0
    if type(obj) == str:
        size = len(obj)
    else:
        size = asizeof.asizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

def group_by_prefix(records):
    prefix_groups = defaultdict(lambda: {'count': 0, 'key_size': 0, 'value_size': 0, 'total_size': 0})

    for record in records:
        if 'ml' in record:
            for key, value in record['ml'].items():
                prefix = key.split(':')[0]
                prefix_groups[prefix]['count'] += 1
                prefix_groups[prefix]['key_size'] += get_size(key)
                prefix_groups[prefix]['value_size'] += get_size(value)
                prefix_groups[prefix]['total_size'] += get_size(key) + get_size(value)

    return prefix_groups

# Example usage
grouped_data = group_by_prefix(cloned_records)
for prefix, data in grouped_data.items():
    print(f"Prefix: {prefix}, Count: {data['count']}, Key Size: {data['key_size']} bytes, Value Size: {data['value_size']} bytes")

Prefix: weather, Count: 3146, Key Size: 77506 bytes, Value Size: 82328 bytes
Prefix: drugsAndSupplements, Count: 17570, Key Size: 662740 bytes, Value Size: 450744 bytes
Prefix: food, Count: 24944, Key Size: 1092972 bytes, Value Size: 707288 bytes
Prefix: phoneUsage, Count: 29457, Key Size: 1677441 bytes, Value Size: 883536 bytes
Prefix: exercise, Count: 16984, Key Size: 688304 bytes, Value Size: 466792 bytes
Prefix: fitbit, Count: 572, Key Size: 19448 bytes, Value Size: 12012 bytes
Prefix: sunrise, Count: 2288, Key Size: 51480 bytes, Value Size: 73216 bytes
Prefix: date, Count: 1716, Key Size: 20592 bytes, Value Size: 46332 bytes
Prefix: events, Count: 2118, Key Size: 49478 bytes, Value Size: 64256 bytes
Prefix: settledScore, Count: 286, Key Size: 3432 bytes, Value Size: 4760 bytes
Prefix: night, Count: 110758, Key Size: 4110315 bytes, Value Size: 2753357 bytes
Prefix: energy, Count: 1499, Key Size: 61311 bytes, Value Size: 36584 bytes
Prefix: sunExposure, Count: 2032, Key Size: 67727 

In [28]:
import pandas as pd

# Convert the grouped data into a DataFrame
grouped_data_df = pd.DataFrame.from_dict(grouped_data, orient='index')

# Reset the index to turn the index into a column
grouped_data_df.reset_index(inplace=True)

# Rename the columns for clarity
grouped_data_df.columns = ['Prefix', 'Count', 'Value Size', 'Key Size', 'Total Size']

# Sort the DataFrame by the 'Size' column in descending order
grouped_data_df = grouped_data_df.sort_values(by='Total Size', ascending=False)

grouped_data_df

,Prefix,Count,Value Size,Key Size,Total Size
10,night,110758,4110315,2753357,6863672
3,phoneUsage,29457,1677441,883536,2560977
2,food,24944,1092972,707288,1800260
4,exercise,16984,688304,466792,1155096
1,drugsAndSupplements,17570,662740,450744,1113484
20,circadian,7450,366351,217680,584031
16,coreBodyTemp,7544,314756,181568,496324
15,hr,6012,190518,176600,367118
18,hrv,5588,182958,163976,346934
21,position,3720,120526,115104,235630


In [32]:
import pandas as pd

# Perform the aggregation
aggregated_data = grouped_data_df.groupby('Prefix').agg(
    {
        'Count': 'sum',
        'Value Size': 'sum',
        'Key Size': 'sum',
        'Total Size': 'sum'
    }
)

# Calculate the Key/Value Ratio
aggregated_data['Key/Value Ratio'] = aggregated_data['Key Size'] / aggregated_data['Value Size']

aggregated_data

,Count,Value Size,Key Size,Total Size,Key/Value Ratio
Prefix,,,,,
bedroomEnvironment,1144,70540,27624,98164,0.391608
circadian,7450,366351,217680,584031,0.594184
coreBodyTemp,7544,314756,181568,496324,0.576853
date,1716,20592,46332,66924,2.250000
drugsAndSupplements,17570,662740,450744,1113484,0.680122
eeg,105,4617,2688,7305,0.582196
energy,1499,61311,36584,97895,0.596696
events,2118,49478,64256,113734,1.298678
exercise,16984,688304,466792,1155096,0.678177


In [30]:
grouped_data_df['Total Size'].sum()

16694685

In [36]:
[(key, cloned_records[-2]['ml'][key]) for key in cloned_records[-2]['ml'].keys() if key.startswith('bedroomEnvi:')]

[('date:month', 1),
 ('date:dayOfWeek', 5),
 ('date:asStr', '2025-01-31'),
 ('date:daysSince', 0),
 ('date:dayOfMonth', 31),
 ('date:year', 2025)]